In [134]:
from keras.layers import Conv3D, MaxPooling3D, ZeroPadding3D, UpSampling3D, Input, Dense, ZeroPadding3D
from keras.models import Model
from keras import backend as K
import numpy as np
from sklearn import preprocessing
from random import randint

In [135]:
# encoder

input_shape = (56, 64, 24, 1)
input_img = Input(shape=input_shape)

# x = ZeroPadding3D(padding=((2, 3),(2, 1),(1, 0)))(input_img)
x = Conv3D(16, (3, 3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling3D((2, 2, 2), padding='same')(x)
x = Conv3D(8, (3, 3, 3), activation='relu', padding='same')(x)

x = MaxPooling3D((2, 2, 2), padding='same')(x)
x = Conv3D(8, (3, 3, 3), activation='relu', padding='same')(x)

encoded = MaxPooling3D((2, 2, 2), padding='same')(x)

In [136]:
# decoder

x = Conv3D(8, (3, 3, 3), activation='relu', padding='same')(encoded)
x = UpSampling3D((2, 2, 2))(x)
x = Conv3D(8, (3, 3, 3), activation='relu', padding='same')(x)
x = UpSampling3D((2, 2, 2))(x)
x = Conv3D(16, (3, 3, 3), activation='relu', padding='same')(x)
x = UpSampling3D((2, 2, 2))(x)
decoded = Conv3D(1, (3, 3, 3), activation='relu', padding='same')(x)

In [146]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_36 (InputLayer)        (None, 56, 64, 24, 1)     0         
_________________________________________________________________
conv3d_147 (Conv3D)          (None, 56, 64, 24, 16)    448       
_________________________________________________________________
max_pooling3d_64 (MaxPooling (None, 28, 32, 12, 16)    0         
_________________________________________________________________
conv3d_148 (Conv3D)          (None, 28, 32, 12, 8)     3464      
_________________________________________________________________
max_pooling3d_65 (MaxPooling (None, 14, 16, 6, 8)      0         
_________________________________________________________________
conv3d_149 (Conv3D)          (None, 14, 16, 6, 8)      1736      
_________________________________________________________________
max_pooling3d_66 (MaxPooling (None, 7, 8, 3, 8)        0         
__________

In [147]:
def train_test_split(split_part):
	og_data = np.load('images.npy')
	labels = np.load('labels.npy')
	
	# Label encoder
	le = preprocessing.LabelEncoder()
	le.fit(labels)
	labels = le.transform(labels)
	x_train, x_test, y_train, y_test = [], [], [], []

	norm_data = og_data/255
	# Sorting the data
	sorted_index = np.argsort(labels)
	sorted_labels = labels[sorted_index]
	sorted_og_data = og_data[sorted_index]
	sorted_norm_data = norm_data[sorted_index]

	# split the data into train and test
	test_random_indices = []
	for i in range(int(sorted_labels.shape[0]/split_part)):
		index = i*split_part + randint(0, split_part - 1)
		test_random_indices.append(index)

	x_train = np.delete(sorted_norm_data, test_random_indices, 0)
	x_test = sorted_norm_data[test_random_indices]
	y_train = np.delete(sorted_labels, test_random_indices)
	y_test = sorted_labels[test_random_indices]
	
	return x_train, x_test, y_train, y_test

In [148]:
x_train, x_test, y_train, y_test = train_test_split(6)

In [149]:
X = []
for i in range(x_train.shape[0]):
    X.append(np.pad(x_train[i], ((3, 2), (2, 1), (1, 0)), 'constant', constant_values=(0.0)))

In [150]:
X = np.array(X, dtype=float)
X = X.reshape(300, 56, 64, 24, 1)

In [151]:
autoencoder.fit(X, X,
                epochs=10,
                batch_size=1,
                verbose=1)

Epoch 1/10
300/300 [==============================] - 416s 1s/step - loss: 0.2113
Epoch 2/10
300/300 [==============================] - 409s 1s/step - loss: 0.2049
Epoch 3/10
 50/300 [====>.........................] - ETA: 6:30 - loss: 0.2008

KeyboardInterrupt: 